# D206 PA code - Doug Haunsperger
### Do initial package import and data read

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('medical_clean.csv')

#view first 5 rows
#df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'medical_clean.csv'

In [ ]:
df_no = df[df[ReAdmis == "No"]]
df_yes = df[df[ReAdmis == "Yes"]]

df_no.info()
df_yes.info()

In [ ]:
df.info()
df.isnull().sum()[df.isnull().sum() > 0]

In [ ]:
df.duplicated().value_counts()

Check to see if there are any duplicate `Customer_id` values, perhaps with different data entered

In [ ]:
df.duplicated(subset = ['Customer_id'], keep = False).value_counts()

Now we notice in the `head()` above that the first two columns are suspiciously the same. Let's check to see if the column is completely duplicated:

In [ ]:
df[df['Unnamed: 0'] == df['CaseOrder']].shape

In [ ]:
df[df['Unnamed: 0'] != df['CaseOrder']].shape
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
# Install package if needed for environment
# !{sys.executable} -m pip install missingno
import missingno as msno
import matplotlib.pyplot as plt
msno.matrix(df, labels=True) # matrix function turns off labels by default when # columns > 50
# See https://github.com/ResidentMario/missingno/issues/93 (Bilogur, 2019)
plt.show()

We see we have missing values in the columns `Children`, `Age`, `Income`, `Soft_drink`, `Overweight`, `Anxiety`, and `Initial_days`. <br>`Children`, `Age`, `Income`, and `Soft_drink` have the most missing values; each have on the order of 25% of the observations missing.

In [ ]:
dict_area = {"Rural": 1, "Suburban": 2, "Urban": 3}
dict_edu = {"No Schooling Completed": 0, "Nursery School to 8th Grade": 8, "9th Grade to 12th Grade, No Diploma": 10, "Regular High School Diploma": 12, 
            "GED or Alternative Credential": 12, "Professional School Degree": 12, "Some College, Less than 1 Year": 12, "Some College, 1 or More Years, No Degree": 13,
            "Associate's Degree": 14, "Bachelor's Degree": 16, "Master's Degree": 18, "Doctorate Degree": 20}
dict_compl = {"Low": 1, "Medium": 2, "High": 3}
dict_yn = {"Yes": 1, "No": 0}
replace_dict = {'Area': dict_area, 'Education': dict_edu, 'ReAdmis': dict_yn, 'Soft_drink': dict_yn, 'HighBlood': dict_yn, 'Stroke': dict_yn, 'Complication_risk':dict_compl, 
                'Arthritis': dict_yn, 'Diabetes': dict_yn, 'Hyperlipidemia': dict_yn, 'BackPain': dict_yn, 'Allergic_rhinitis': dict_yn, 'Reflux_esophagitis': dict_yn, 'Asthma': dict_yn}
# Overweight, Anxiety, and Item[1-8] are already expressed ordinally
# Make copy to keep original df
df_orig = df.copy(deep = True)
df.replace(replace_dict, inplace = True)

In [ ]:
#df['Child_missing']=df['Children'].isna()
#df['Age_missing']=df['Age'].isna()
#df['Income_missing']=df['Income'].isna()
#df['SD_missing']=df['Soft_drink'].isna()
#df['OW_missing']=df['Overweight'].isna()
#df['Anx_missing']=df['Anxiety'].isna()
#df['ID_missing']=df['Initial_days'].isna()
#Reduce dataset to remove survey, ID variables
dfr = df.iloc[:,13:]
dfr = dfr[[c for c in dfr.columns if not c.startswith('Item')]]
#dfr = dfr[[c for c in dfr.columns if not c in dfr.iloc[:,11:30].columns]]
dfr.info()

In [ ]:
msno.matrix(dfr.sort_values('Diabetes'))

In [ ]:
import seaborn as sns
sns.histplot(data=dfr,x='TotalCharge',hue="Gender")

In [ ]:
sns.pairplot(dfr)

In [ ]:
sns.pairplot(df[['Age','Additional_charges','TotalCharge']])

In [ ]:
print(df.loc[(df['VitD_levels'] > 35), 'State'].unique())

In [ ]:
print(df.loc[(df['VitD_levels'] < 35), 'State'].unique())

In [ ]:
df['high_line'] = (df['Additional_charges']/(df['Age']+20) > 150)

In [ ]:
sns.pairplot(df[['Age','Additional_charges','HighBlood','high_line']], hue='high_line')

In [ ]:
sns.pairplot(dfr,hue='high_line')

In [ ]:
df[(df['HighBlood'] == 0) & (df['high_line'] == True)].shape